In [144]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
import numpy as np

In [145]:
embeds = [
    [0.43, 0.15, 0.89], # Your 
    [0.55, 0.87, 0.66], # journey
    [0.57, 0.85, 0.64], # starts
    [0.22, 0.58, 0.33], # with
    [0.77, 0.25, 0.10], # one
    [0.05, 0.80, 0.55]  # step
]

In [146]:
#
#
#

In [147]:
pip install -q torch

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# import torch
# import torch.nn as nn
# torch.manual_seed(123)
# d_in = 3
# d_out = 2
# q_weight     = nn.Parameter(torch.rand(d_in, d_out))
# key_weight   = nn.Parameter(torch.rand(d_in, d_out))
# value_weight = nn.Parameter(torch.rand(d_in, d_out))

# print("q_weight:\n", q_weight)

# print("key_weight:\n", key_weight)

# print("value_weight:\n", value_weight)

In [186]:
import torch

inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)
import torch.nn as nn

class SelfAttention_v1(nn.Module):

    def __init__(self, d_in, d_out):
        super().__init__()
        self.W_query = nn.Parameter(torch.rand(d_in, d_out))
        self.W_key   = nn.Parameter(torch.rand(d_in, d_out))
        self.W_value = nn.Parameter(torch.rand(d_in, d_out))

    def forward(self, x):
        keys = x @ self.W_key
        # print("keys:", keys)
        queries = x @ self.W_query
        # print("queries:", queries)
        values = x @ self.W_value
        # print("values:", values) 
        
        attn_scores = queries @ keys.T # omega
        # print("attn_scores:", attn_scores) 
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1
        )
        # print("attn_weights:", attn_weights) 
        context_vec = attn_weights @ values
        return context_vec

torch.manual_seed(123)
sa_v1 = SelfAttention_v1(d_in, d_out)
print(sa_v1(inputs))

tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]], grad_fn=<MmBackward0>)


In [187]:
# sa_v1.W_query
# sa_v1.W_key
# sa_v1.W_value

In [115]:
#
#
#
#

In [155]:
inputs = tf.constant(embeds)
query = inputs[1]
attn_scores_2 = np.empty(inputs.shape[0])

for i, x_i in enumerate(inputs):
    print(i, x_i)
    attn_scores_2[i] = x_i.numpy() @ query.numpy()
attn_scores_2 = tf.constant(attn_scores_2, dtype=tf.float32)    
print("attn_scores_2:", attn_scores_2)

0 tf.Tensor([0.43 0.15 0.89], shape=(3,), dtype=float32)
1 tf.Tensor([0.55 0.87 0.66], shape=(3,), dtype=float32)
2 tf.Tensor([0.57 0.85 0.64], shape=(3,), dtype=float32)
3 tf.Tensor([0.22 0.58 0.33], shape=(3,), dtype=float32)
4 tf.Tensor([0.77 0.25 0.1 ], shape=(3,), dtype=float32)
5 tf.Tensor([0.05 0.8  0.55], shape=(3,), dtype=float32)
attn_scores_2: tf.Tensor([0.95440006 1.4950001  1.4754001  0.8434     0.707      1.0865    ], shape=(6,), dtype=float32)


In [123]:
# attn_weights_2 is softmax of attn_scores_2
attn_weights_2 = tf.nn.softmax(attn_scores_2)
print("attn_weights_2:", attn_weights_2)

attn_weights_2: tf.Tensor([0.13854761 0.23789133 0.23327404 0.1239916  0.10818188 0.15811363], shape=(6,), dtype=float32)


In [124]:
inputs

<tf.Tensor: shape=(6, 3), dtype=float32, numpy=
array([[0.43, 0.15, 0.89],
       [0.55, 0.87, 0.66],
       [0.57, 0.85, 0.64],
       [0.22, 0.58, 0.33],
       [0.77, 0.25, 0.1 ],
       [0.05, 0.8 , 0.55]], dtype=float32)>

In [125]:

context_vec2 = tf.tensordot(attn_weights_2, inputs, axes=1)
context_vec2

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([0.44186577, 0.65148205, 0.56830895], dtype=float32)>

In [126]:
#
#
#
#

In [127]:
class SelfAttenion_v0(tf.keras.layers.Layer):
    def __init__(self):
        super().__init__()
        pass
    
    def __call__(self, x):
        keys = x
        queries = x
        values = x

        attn_scores = tf.matmul(queries, keys, transpose_b=True)
        attn_weights = tf.nn.softmax(attn_scores)
        all_context_vecs = tf.matmul(attn_weights, values, transpose_b=False)
        return all_context_vecs

In [128]:
sa = SelfAttenion_v0()
sa(inputs)

<tf.Tensor: shape=(6, 3), dtype=float32, numpy=
array([[0.44205937, 0.5930985 , 0.578989  ],
       [0.4418658 , 0.65148205, 0.568309  ],
       [0.44312754, 0.6495946 , 0.5670731 ],
       [0.43038973, 0.6298281 , 0.55102706],
       [0.46710175, 0.5909928 , 0.5265966 ],
       [0.41772443, 0.65032315, 0.56453526]], dtype=float32)>

In [129]:
#
#
#


In [41]:
# weights from LLMFS listing 3.1
qw = np.array([[0.2961, 0.5166],
        [0.2517, 0.6886],
        [0.0740, 0.8665]], dtype=np.float32)
kw = np.array([[0.1366, 0.1025],
        [0.1841, 0.7264],
        [0.3153, 0.6871]], dtype=np.float32)
vw = np.array([[0.0756, 0.1966],
        [0.3164, 0.4017],
        [0.1186, 0.8274]], dtype=np.float32)
# dummy_input = tf.zeros(shape=(6, 3))


In [182]:
# Corresponds to LLMFS listing 3.2
class SelfAttenion_v1(tf.keras.layers.Layer):
    def __init__(self):
        super().__init__()
        b = 0
        self.query_layer = tf.keras.layers.Dense(units=2, activation=None, name=f"query-{b}", use_bias=False)
        self.query_layer.build((None, 3 ))
        self.key_layer = tf.keras.layers.Dense(units=2, activation=None, name=f"key-{b}", use_bias=False)
        self.key_layer.build((None, 3))
        self.value_layer = tf.keras.layers.Dense(units=2, activation=None, name=f"value-{b}", use_bias=False)
        self.value_layer.build((None, 3))
    def __call__(self, x):
        keys = self.key_layer(x)
        # print("keys:", keys)
        queries = self.query_layer(x)
        # print("queries:", queries)
        values = self.value_layer(x)
        # print("values:", values)
        attn_scores = tf.matmul(queries, keys, transpose_b=True)
        # print("attn_scores:", attn_scores)
        attn_weights = tf.nn.softmax(attn_scores / keys.shape[-1]**0.5, axis=-1)
        # print("attn_weights:", attn_weights)
        all_context_vecs = tf.matmul(attn_weights, values, transpose_b=False)
        return all_context_vecs

In [188]:
sa = SelfAttenion_v1()
sa.query_layer.set_weights([qw])
sa.key_layer.set_weights([kw])
sa.value_layer.set_weights([vw])
# sa.query_layer.get_config()

# sa.query_layer(dummy_input)
# sa.query_layer.set_weights([qw])
# sa.key_layer.set_weights([kw])
# sa.value_layer.set_weights([vw])

In [189]:
inputs = tf.constant(embeds)
sa(inputs)

<tf.Tensor: shape=(6, 2), dtype=float32, numpy=
array([[0.2995769 , 0.8052748 ],
       [0.30609584, 0.8209922 ],
       [0.30577675, 0.82025766],
       [0.29476127, 0.7938287 ],
       [0.29270154, 0.7890473 ],
       [0.29900536, 0.8039986 ]], dtype=float32)>

In [228]:
import tensorflow as tf

def create_causal_mask(seq_len):
    """
    Generates a boolean causal mask of shape (seq_len, seq_len).
    """
    # Create a tensor of shape (seq_len, seq_len)
    mask = tf.ones((seq_len, seq_len), dtype=tf.bool)
    
    # Use band_part to keep only the lower triangle (including the diagonal)
    # num_lower: -1 means keep all lower-triangular parts
    # num_upper: 0 means keep 0 upper-triangular parts (only the diagonal)
    lower_triangular = tf.linalg.band_part(
        mask, 
        num_lower=-1, 
        num_upper=0
    )
    
    return lower_triangular

In [231]:
causal_mask = create_causal_mask(6)
additive_mask = 1.0 - tf.cast(causal_mask, dtype=tf.float32)
additive_mask
large_negative_value = -1e9 
additive_mask_applied = additive_mask * large_negative_value
additive_mask_applied

<tf.Tensor: shape=(6, 6), dtype=float32, numpy=
array([[-0.e+00, -1.e+09, -1.e+09, -1.e+09, -1.e+09, -1.e+09],
       [-0.e+00, -0.e+00, -1.e+09, -1.e+09, -1.e+09, -1.e+09],
       [-0.e+00, -0.e+00, -0.e+00, -1.e+09, -1.e+09, -1.e+09],
       [-0.e+00, -0.e+00, -0.e+00, -0.e+00, -1.e+09, -1.e+09],
       [-0.e+00, -0.e+00, -0.e+00, -0.e+00, -0.e+00, -1.e+09],
       [-0.e+00, -0.e+00, -0.e+00, -0.e+00, -0.e+00, -0.e+00]],
      dtype=float32)>

In [232]:
# Corresponds to LLMFS listing 3.3
class CausalAttention(tf.keras.layers.Layer):
    def __init__(self):
        super().__init__()
        b = 0
        self.query_layer = tf.keras.layers.Dense(units=2, activation=None, name=f"query-{b}", use_bias=False)
        self.query_layer.build((None, 3 ))
        self.key_layer = tf.keras.layers.Dense(units=2, activation=None, name=f"key-{b}", use_bias=False)
        self.key_layer.build((None, 3))
        self.value_layer = tf.keras.layers.Dense(units=2, activation=None, name=f"value-{b}", use_bias=False)
        self.value_layer.build((None, 3))
    def __call__(self, x):
        # b, 
        num_tokens, d_in = x.shape
        keys = self.key_layer(x)
        # print("keys:", keys)
        queries = self.query_layer(x)
        # print("queries:", queries)
        values = self.value_layer(x)
        # print("values:", values)
        # attn_scores = tf.matmul(queries, keys, transpose_b=True)
        # coefficients = tf.matmul(queries, tf.transpose(keys, perm=[0,2,1]))
        coefficients = tf.matmul(queries, tf.transpose(keys, perm=[1,0]))
        print("coefficients:", coefficients)

        # mask = tf.linalg.band_part(tf.ones((num_tokens, num_tokens)), -1, 0)
        # print("mask:", mask)
        attn_scores = coefficients + additive_mask_applied
        print("attn_scores:", attn_scores)
        attn_weights = tf.nn.softmax(attn_scores / keys.shape[-1]**0.5, axis=-1)
        # print("attn_weights:", attn_weights)
        all_context_vecs = tf.matmul(attn_weights, values, transpose_b=False)
        return all_context_vecs

In [233]:
inputs = tf.constant(embeds)
batch = tf.stack((inputs, inputs))
batch

<tf.Tensor: shape=(2, 6, 3), dtype=float32, numpy=
array([[[0.43, 0.15, 0.89],
        [0.55, 0.87, 0.66],
        [0.57, 0.85, 0.64],
        [0.22, 0.58, 0.33],
        [0.77, 0.25, 0.1 ],
        [0.05, 0.8 , 0.55]],

       [[0.43, 0.15, 0.89],
        [0.55, 0.87, 0.66],
        [0.57, 0.85, 0.64],
        [0.22, 0.58, 0.33],
        [0.77, 0.25, 0.1 ],
        [0.05, 0.8 , 0.55]]], dtype=float32)>

In [234]:
ca = CausalAttention()

In [235]:
ca(inputs)

coefficients: tf.Tensor(
[[-0.19195148 -0.36451432 -0.38414305 -0.14244172 -0.62964225  0.03199355]
 [-0.33032215 -0.6104431  -0.63550246 -0.2594384  -0.90808207 -0.04208406]
 [-0.32210398 -0.5963464  -0.6213471  -0.25205636 -0.8968554  -0.03474403]
 [-0.20046927 -0.36579722 -0.37858456 -0.16142553 -0.50239176 -0.05251247]
 [-0.08335634 -0.17448942 -0.19140114 -0.04808448 -0.4422074   0.10734363]
 [-0.29117894 -0.5230913  -0.5374055  -0.24146162 -0.6440127  -0.12372647]], shape=(6, 6), dtype=float32)
attn_scores: tf.Tensor(
[[-1.9195148e-01 -1.0000000e+09 -1.0000000e+09 -1.0000000e+09
  -1.0000000e+09 -1.0000000e+09]
 [-3.3032215e-01 -6.1044312e-01 -1.0000000e+09 -1.0000000e+09
  -1.0000000e+09 -1.0000000e+09]
 [-3.2210398e-01 -5.9634638e-01 -6.2134713e-01 -1.0000000e+09
  -1.0000000e+09 -1.0000000e+09]
 [-2.0046927e-01 -3.6579722e-01 -3.7858456e-01 -1.6142553e-01
  -1.0000000e+09 -1.0000000e+09]
 [-8.3356343e-02 -1.7448942e-01 -1.9140114e-01 -4.8084475e-02
  -4.4220740e-01 -1.0000000e

<tf.Tensor: shape=(6, 2), dtype=float32, numpy=
array([[-0.7828562 ,  0.42228222],
       [-0.9191122 ,  0.3110891 ],
       [-0.9676956 ,  0.2567558 ],
       [-0.86749935,  0.22659169],
       [-0.83010334,  0.08190385],
       [-0.79709584,  0.17433873]], dtype=float32)>